In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential,Model
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import openpyxl
from modules import *
%matplotlib inline #Contain: stride, batch_size, nPosSoftSize, data_preparation(), decoder(), encoder()

2021-08-29 14:12:48.339747: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
UsageError: unrecognized arguments: #Contain: stride, batch_size, nPosSoftSize, data_preparation(), decoder(), encoder()


In [2]:
img_train,label_train,img_validation,_,label_validation_=data_preparation()

img_train = np.append(img_train, [np.fliplr(x) for x in img_train], axis=0)
img_train = np.append(img_train, [np.flipud(x) for x in img_train], axis=0)
label_train = np.append(label_train, [np.fliplr(x) for x in label_train], axis=0)
label_train = np.append(label_train, [np.flipud(x) for x in label_train], axis=0)

print(img_train.shape)
print(label_train.shape)
print(img_validation.shape)
print(label_validation_.shape)

(980, 128, 128, 3)
(980, 128, 128, 1)
(147, 128, 128, 3)
(3, 512, 512, 1)


In [3]:
encoder = encoder()
encoder.compile(optimizer = 'adam', 
                    loss = 'mse', 
                    metrics = ['accuracy'])
encoder.summary()

2021-08-29 14:12:50.512288: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-29 14:12:50.512734: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-29 14:12:50.541441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 14:12:50.541831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.68GHz coreCount: 34 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2021-08-29 14:12:50.541849: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-08-29 14:12:50.543274: I tensorflow/stream_executor/platform/defa

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 128, 128, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
____________________________________________________________________________________________

In [4]:
decoder = decoder()
decoder.load_weights('weights/decoder_weights_best.h5')

In [5]:
layer_input = tf.keras.layers.Input(shape=(128,128,1))
x = layer_input
for layer in decoder.layers[1:5]:
    x = layer(x)
#ED:Encoder part of decoder network
ED = tf.keras.Model(layer_input, x)

#code:target 1/16 feature maps
code=ED.predict(label_train)
label_train=None

2021-08-29 14:12:52.772947: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-29 14:12:52.793172: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3799900000 Hz
2021-08-29 14:12:52.851870: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-08-29 14:12:53.791568: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-08-29 14:12:54.005760: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 14:12:54.022190: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 14:12:54.037278: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 14:12

In [6]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(img_train,
                                     code,
                                     batch_size = batch_size) 

img_train=None
code=None

In [7]:
layer_input = tf.keras.layers.Input(shape=(8,8,32))
label = layer_input
for layer in decoder.layers[5:]:
    label = layer(label)
#DD:Decoder part of decoder network
DD = tf.keras.Model(layer_input, label)
DD.summary()
decoder=None

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8, 8, 32)]        0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 16, 16, 32)        25728     
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 32, 32)        25728     
_________________________________________________________________
sequential_6 (Sequential)    (None, 64, 64, 16)        12864     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 128, 128, 1)       401       
Total params: 64,721
Trainable params: 64,561
Non-trainable params: 160
_________________________________________________________________


In [8]:
def METRICS(pre_val,label_val):
    """
    Performance evaluation after every epoch
    nPosSoftSize:buffer size
    tp:true-positive pixels
    fp:false-positive pixels
    fn:false-negative pixels
    """
    global nPosSoftSize
    kernel = np.ones([nPosSoftSize*2+1,nPosSoftSize*2+1])
    def metrics(res,label):
        res=res[:,:,0]
        label=label[:,:,0]

        res_dilate=cv2.dilate(res,kernel,1)
        label_dilate=cv2.dilate(label,kernel,1)

        ones_like = tf.ones_like(res)
        zeros_like = tf.zeros_like(res)

        tp = tf.cast(tf.logical_and(tf.equal(res, ones_like),tf.equal(label_dilate, ones_like)),"float")
        fp = tf.cast(tf.logical_and(tf.equal(res, ones_like),tf.equal(label_dilate, zeros_like)),"float")
        precision=tf.reduce_sum(tp)/(tf.reduce_sum(tp)+tf.reduce_sum(fp)+1e-20)

        fn = tf.cast(tf.logical_and(tf.equal(res_dilate, zeros_like),tf.equal(label, ones_like)),"float") 
        recall=tf.reduce_sum(tp)/(tf.reduce_sum(tp)+tf.reduce_sum(fn)+1e-20)
        F1=2*precision*recall/(precision+recall+1e-20)

        precision=np.array(precision)
        recall=np.array(recall)
        F1=np.array(F1)

        return precision,recall,F1
    
    precision=0
    recall=0
    F1=0
    for i in range(pre_val.shape[0]):
        A,B,C=metrics(pre_val[i],label_val[i])
        precision=precision+A
        recall=recall+B
        F1=F1+C
    precision=precision/pre_val.shape[0]
    recall=recall/pre_val.shape[0]
    F1=F1/pre_val.shape[0]
    return precision,recall,F1

In [9]:
F_M=0.0
EPOCH=1
workbook = openpyxl.Workbook()
worksheet = workbook.create_sheet(index=0)

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        '''
        EPOCH: Current epoch
        F_M: Maximum F1-score
        stride: Stride of the sliding window
        H,W: Height and witdh of the original images
        h,w: Height and witdh of the network inputs
        '''
        global F_M
        global EPOCH
        global stride
        code=encoder.predict(img_validation)
        pre_validation=np.round(DD.predict(code))
        pre_validation_=np.zeros(label_validation_.shape)
        H = label_validation_.shape[1]
        W = label_validation_.shape[2]
        h = 128
        w = 128
        stride = stride 
        #perdiction assemble
        for i in range(pre_validation_.shape[0]):
            for k1 in range(int((H-h)/ stride)+1):
                for k2 in range(int((W-w)/ stride)+1):
                    pre_validation_[i,k1*stride:k1*stride+h,k2*stride:k2*stride+w]+=pre_validation[i*(int((H-h)/ stride)+1)*(int((W-w)/ stride)+1)+k1*(int((W-w)/ stride)+1)+k2] 
        pre_validation_=np.sign(pre_validation_)  
        precision,recall,F1=METRICS(pre_validation_,label_validation_)

        print('\n', "precision:", precision, '\n', "recall:", recall, '\n', "F1 :",  F1)
        
        worksheet.cell(EPOCH,1).value=precision
        worksheet.cell(EPOCH,2).value=recall
        worksheet.cell(EPOCH,3).value=F1
        workbook.save("history/encoder_history.xlsx")
        
        encoder.save_weights('weights/encoder_weights.h5')
        if F1>F_M:
            F_M=F1
            encoder.save_weights('weights/encoder_weights_best.h5')
            
        EPOCH=EPOCH+1

In [10]:
history = encoder.fit(
          train_generator,
          epochs = 100,
          verbose = 1,
          callbacks=[MyCustomCallback()])

Epoch 1/100


2021-08-29 02:54:18.159483: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.174600: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.195212: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.202627: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.213835: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.232021: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.239072: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-0

 1/31 [..............................] - ETA: 1:08 - loss: 1.7222 - accuracy: 0.0359

2021-08-29 02:54:18.604092: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.611690: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.626981: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.635625: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.675214: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.686279: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:18.705157: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-0

 5/31 [===>..........................] - ETA: 4s - loss: 1.7916 - accuracy: 0.0584

2021-08-29 02:54:19.224736: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED


31/31 [==============================] - 4s 69ms/step - loss: 1.5064 - accuracy: 0.1203


2021-08-29 02:54:21.094189: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.103401: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.119354: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.125853: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.134994: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.149819: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-29 02:54:21.155442: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-0


 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 2/100
31/31 [==============================] - 2s 51ms/step - loss: 0.8504 - accuracy: 0.2588

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 3/100
31/31 [==============================] - 2s 52ms/step - loss: 0.5718 - accuracy: 0.3716

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 4/100
31/31 [==============================] - 2s 49ms/step - loss: 0.4257 - accuracy: 0.3980

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 5/100
31/31 [==============================] - 2s 49ms/step - loss: 0.3191 - accuracy: 0.4314 0s - loss: 0.3197 - accu

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 6/100
31/31 [==============================] - 2s 51ms/step - loss: 0.2584 - accuracy: 0.3696

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 7/100
31/31 [==============================] - 2s 51ms/step - loss: 0.2133 - accuracy: 0.4175

 precision: 0.0 
 recall: 0.0 
 F1 : 0.0
Epoch 8/100
31/31 [==============================] - 2s 51ms/step - loss: 0.1710 

Epoch 94/100
31/31 [==============================] - 2s 50ms/step - loss: 0.0167 - accuracy: 0.7202

 precision: 0.9679996371269226 
 recall: 0.9433822433153788 
 F1 : 0.9545338352521261
Epoch 95/100
31/31 [==============================] - 2s 50ms/step - loss: 0.0164 - accuracy: 0.7730

 precision: 0.9644876917203268 
 recall: 0.9541501005490621 
 F1 : 0.9584404230117798
Epoch 96/100
31/31 [==============================] - 2s 51ms/step - loss: 0.0156 - accuracy: 0.7242

 precision: 0.9571648041407267 
 recall: 0.9618371923764547 
 F1 : 0.9589001536369324
Epoch 97/100
31/31 [==============================] - 2s 52ms/step - loss: 0.0132 - accuracy: 0.7764

 precision: 0.958805779616038 
 recall: 0.957669715086619 
 F1 : 0.9575455586115519
Epoch 98/100
31/31 [==============================] - 2s 52ms/step - loss: 0.0172 - accuracy: 0.7882

 precision: 0.9715967774391174 
 recall: 0.9433348377545675 
 F1 : 0.9561687906583151
Epoch 99/100
31/31 [==============================] - 2s 52ms/